In [2]:
import pandas as pd
import os
import numpy as np
import logging
from tqdm import tqdm

# Configuring the logger
logging.basicConfig(filename='Data/IndicatorData/_IndicatorStatProperties.log',
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Path to the directory containing files
input_directory = 'Data/IndicatorData'

# Function to calculate statistical properties of a column
def calculate_stats(df, column):
    stats = {
        'Mean': df[column].mean(),
        'StdDev': df[column].std(),
        'Min': df[column].min(),
        '25th Percentile': df[column].quantile(0.25),
        'Median': df[column].median(),
        '75th Percentile': df[column].quantile(0.75),
        'Max': df[column].max(),
        'Kurtosis': df[column].kurtosis(),
        'Skew': df[column].skew()
    }
    return stats

def process_files_and_log_stats(input_dir):
    files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')]
    stats_dict = {}

    for file in tqdm(files, desc="Processing Files"):
        try:
            df = pd.read_csv(file)
            for column in df.columns:
                # Check if the column is numerical
                if df[column].dtype in [np.float64, np.int64, np.float32, np.int32]:
                    column_stats = calculate_stats(df, column)
                    if column not in stats_dict:
                        stats_dict[column] = []
                    stats_dict[column].append(column_stats)
        except Exception as e:
            logging.error(f"Error processing {file}: {e}")

    # Calculate average stats for each column
    avg_stats = {col: pd.DataFrame(stats).mean().to_dict() for col, stats in stats_dict.items()}

    # Convert to DataFrame
    avg_stats_df = pd.DataFrame(avg_stats).transpose()
    avg_stats_df = avg_stats_df.round(4)
    return avg_stats_df

# Run the function
avg_stats_df = process_files_and_log_stats(input_directory)

# Write to CSV
avg_stats_df.to_csv('__ScalingStats.csv', index_label='Column')


Processing Files:   0%|          | 0/1759 [00:00<?, ?it/s]/root/root/miniconda4/envs/tf/lib/python3.9/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/root/root/miniconda4/envs/tf/lib/python3.9/site-packages/pandas/core/nanops.py:1344: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/root/root/miniconda4/envs/tf/lib/python3.9/site-packages/pandas/core/nanops.py:1256: RuntimeWarning: invalid value encountered in subtract
  adjusted = values - mean
/root/root/miniconda4/envs/tf/lib/python3.9/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
Processing Files:   0%|          | 2/1759 [00:00<02:00, 14.54it/s]/root/root/miniconda4/envs/tf/lib/python3.9/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_

KeyboardInterrupt: 